In [1]:
import pandas as pd
from matplotlib import colors
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
import numpy as np

# 定义文件路径
file_path = 'vdjdb.txt'  # 将 'your_file.txt' 替换为你的文件路径

# 读取文件内容
with open(file_path, 'r', encoding='utf-8') as file:
    # 读取文件的第一行，获取所有的信息变量名
    header = file.readline().strip().split('\t')
    tcr_data = [dict(zip(header, line.strip().split('\t'))) for line in file]
cdr3_dict = {}
for row in tcr_data:
    complex_id = row['complex.id']
    cdr3 = row['cdr3']
    # Splice together CDR3 with the same complex ID
    if complex_id in cdr3_dict:
        cdr3_dict[complex_id].append(cdr3)
    else:
        cdr3_dict[complex_id] = [cdr3]
# There is a DataFrame containing the TCR sequence
for row in tcr_data:
    complex_id = row['complex.id']
    antigen_epitope = row['antigen.epitope']
    vdjdb_score = row['vdjdb.score']
    # Splice together CDR3 with the same complex ID
    if len(cdr3_dict[complex_id]) == 2:
        cdr3_dict[complex_id].append(antigen_epitope)
        cdr3_dict[complex_id].append(vdjdb_score)
    else:
        continue
cdr3_dict.pop('0')
##Delete unpaired TCRs
df_cdr3 = pd.DataFrame(cdr3_dict)
df_cdr3_trans = df_cdr3.transpose()
names = ['TRA', 'TRB', 'antigen_epitope', 'vdjdb.score']
df_cdr3_trans.columns = names
print(df_cdr3_trans)
##The first step is to read out the paired data

                  TRA                   TRB antigen_epitope vdjdb.score
1       CIVRAPGRADMRF  CASSYLPGQGDHYSNQPQHF        FLKEKGGL           2
2      CAVPSGAGSYQLTF   CASSFEPGQGFYSNQPQHF        FLKEKGGL           2
3         CAVKASGSRLT  CASSYEPGQVSHYSNQPQHF        FLKEKGGL           2
4       CAYRPPGTYKYIF        CASSALASLNEQFF        FLKEKGGL           2
5       CIVRAPGRADMRF  CASSYLPGQGDHYSNQPQHF        FLKEQGGL           2
...               ...                   ...             ...         ...
30590   CMDEGGSNYKLTF         CASSVRSTDTQYF    PQPELPYPQPQL           0
30591     CSLYNNNDMRF         CASSLRYTDTQYF    PQPELPYPQPQL           0
30592   CALSTDSWGKLQF       CASSPGQGGDNEQFF   PQQPFPQPEQPFP           0
30593    CAPQGATNKLIF       CASSLGAGGQETQYF   PQQPFPQPEQPFP           2
30594  CLVGGSGGYNKLIF         CASSSTAQETQYF   PQQPFPQPEQPFP           0

[30594 rows x 4 columns]


In [2]:
##clean the data
df_clean = df_cdr3_trans[df_cdr3_trans['vdjdb.score'] != '0']
df_clean['TRA_TRB_Combined'] = df_clean["TRA"] + df_clean["TRB"]
df_clean = df_clean.reset_index(drop=True)
print("There are {} categories of data in the current dataset".format(np.shape(df_clean['antigen_epitope'].unique())))
print(df_clean)

There are (391,) categories of data in the current dataset
                    TRA                   TRB antigen_epitope vdjdb.score  \
0         CIVRAPGRADMRF  CASSYLPGQGDHYSNQPQHF        FLKEKGGL           2   
1        CAVPSGAGSYQLTF   CASSFEPGQGFYSNQPQHF        FLKEKGGL           2   
2           CAVKASGSRLT  CASSYEPGQVSHYSNQPQHF        FLKEKGGL           2   
3         CAYRPPGTYKYIF        CASSALASLNEQFF        FLKEKGGL           2   
4         CIVRAPGRADMRF  CASSYLPGQGDHYSNQPQHF        FLKEQGGL           2   
...                 ...                   ...             ...         ...   
2960         CIALNARLMF         CASSLRATDTQYF    PQPELPYPQPQL           2   
2961   CAMREGRYSSASKIIF       CATSRAGGGGEKLFF    FPQPEQPFPWQP           2   
2962   CLVGDGDGGATNKLIF        CASSQGSGGNEQFF    FPQPEQPFPWQP           2   
2963  CAASVLYGSSNTGKLIF      CASSIVGSGGYNEQFF    QLQPFPQPELPY           2   
2964       CAPQGATNKLIF       CASSLGAGGQETQYF   PQQPFPQPEQPFP           2   

                

/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [3]:
##Edit Distance
def ED(str_1, str_2):
    m = len(str_1)
    n = len(str_2)
    # Initializes the dynamic programming matrix with sizes m+1 and n+1 respectively
    Distance = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    for i in range(n + 1):
        Distance[0][i] = i
    #
    for j in range(m + 1):
        Distance[j][0] = j
    # Initialize the first row and column of the matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            distance_delete = Distance[i - 1][j] + 1
            distance_add = Distance[i][j - 1] + 1
            if str_1[i - 1] == str_2[j - 1]:
                distance_change = Distance[i - 1][j - 1]
            else:
                distance_change = Distance[i - 1][j - 1] + 1
            Distance[i][j] = min(distance_delete, distance_add, distance_change)
    # Count the items from bottom to top
    return Distance[m][n]

In [4]:
##Jaccard Distance
def jaccard_distance(str1, str2):
    set1 = set(str1)
    set2 = set(str2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return 1 - intersection / union

In [ ]:
Distance_Matrix_TRA = np.zeros((df_clean.shape[0], df_clean.shape[0]))
Distance_Matrix_TRB = np.zeros((df_clean.shape[0], df_clean.shape[0]))
Distance_Matrix_TRAandTRB = np.zeros((df_clean.shape[0], df_clean.shape[0]))
for i in range(df_clean.shape[0]):
    for j in range(df_clean.shape[0]):
        Distance_Matrix_TRA[i][j] = jaccard_distance(df_clean['TRA'][i], df_clean['TRA'][j])
        Distance_Matrix_TRB[i][j] = jaccard_distance(df_clean['TRB'][i], df_clean['TRB'][j])
        Distance_Matrix_TRAandTRB[i][j] = jaccard_distance(df_clean['TRA_TRB_Combined'][i], df_clean['TRA_TRB_Combined'][j])
    if i % 10 == 0:
        print("Currently calculated{:2f}TCR3".format(i))

Currently calculated0.000000TCR3
Currently calculated10.000000TCR3
Currently calculated20.000000TCR3
Currently calculated30.000000TCR3
Currently calculated40.000000TCR3
Currently calculated50.000000TCR3
Currently calculated60.000000TCR3
Currently calculated70.000000TCR3
Currently calculated80.000000TCR3
Currently calculated90.000000TCR3
Currently calculated100.000000TCR3
Currently calculated110.000000TCR3
Currently calculated120.000000TCR3
Currently calculated130.000000TCR3
Currently calculated140.000000TCR3
Currently calculated150.000000TCR3
